In [1]:
import pandas as pd
import os
from netCDF4 import Dataset

In [2]:
iso3 = pd.read_csv("ciso3.txt", sep=" ")

In [3]:
iso3 = iso3.set_index('iso3')

In [4]:
results = [i for i in os.listdir('FSC_Scenarios') if 'Results' in i]

In [5]:
results

['NAF_High_Results.csv',
 'MBBF_High_Results.csv',
 'MBBF_Low_Results.csv',
 'CPA_Low_Results.csv',
 'NAF_Mid_Results.csv',
 'FSU_High_Results.csv',
 'FSU_Mid_Results.csv',
 'EUR_Low_Results.csv',
 'CPA_High_Results.csv',
 'FSU_Low_Results.csv',
 'EUR_Mid_Results.csv',
 'NAF_Low_Results.csv',
 'CPA_Mid_Results.csv',
 'MBBF_Mid_Results.csv',
 'EUR_High_Results.csv']

In [6]:
first = True
for r in results:
    region = r.split("_")[0]
    shock = r.split("_")[1]
    if first:
        df = pd.read_csv(f"FSC_Scenarios/{r}")
        df["Region"] = region
        df["Shock"] = shock
        first = False
    else:
        df_ = pd.read_csv(f"FSC_Scenarios/{r}")
        df_["Region"] = region
        df_["Shock"] = shock
        df = df.append(df_)

In [7]:
df['iso3'] = df['Unnamed: 0']

In [8]:
df = df.join(iso3, how="left", on="iso3")
df["Country"] = df["name"]

In [9]:
df.loc[df['iso3']=='SSD', 'Country'] = 'South Sudan'

In [10]:
del(df["Unnamed: 0"])
del(df["FAO"])
del(df["name"])

In [ ]:
df.to_csv("FSC.csv", index=False)

## TWIST Ingestion

In [104]:
results = [i for i in os.listdir('TWIST-Scenarios/') if '.csv' in i]

In [105]:
results

['priceReg_China.csv',
 'priceReg_EUR.csv',
 'priceReg_EUR-China.csv',
 'priceReg_China-Former_UDSSR.csv',
 'priceReg_EUR-Former_UDSSR.csv',
 'priceReg_Former_UDSSR-USMCA-China.csv',
 'priceReg_Former_UDSSR.csv',
 'priceReg_global.csv',
 'priceReg_EUR-Former_UDSSR-China.csv',
 'priceReg_EUR-Former_UDSSR-USMCA.csv',
 'priceReg_EUR-USMCA.csv',
 'priceReg_USMCA-China.csv',
 'priceReg_USMCA-Former_UDSSR.csv',
 'priceReg_USMCA.csv']

In [106]:
results = [i for i in results if '-' not in i]

In [107]:
results

['priceReg_China.csv',
 'priceReg_EUR.csv',
 'priceReg_Former_UDSSR.csv',
 'priceReg_global.csv',
 'priceReg_USMCA.csv']

In [112]:
region_lookup = {"China": "Centrally Planned Asia",
                 "EUR": "European Union",
                 "global": "Multiple Breadbasket Failure",
                 "USMCA": "North Atlantic Free Trade",
                 "Former_UDSSR": "Former Soviet Union"}

In [113]:
pairs = [
    ['time','Baseline_price'],
    ['time','High_shock_price','classification_high_spike'],
    ['time','Medium_shock_price','classification_medium_spike'],
    ['time','Low_shock_price','classification_low_spike']
]

In [120]:
df_out = pd.DataFrame()
for r in results:
    try:
        region = region_lookup[r.split("_")[1].split(".csv")[0]]
    except:
        region = region_lookup["Former_UDSSR"]
    df = pd.read_csv(f'TWIST-Scenarios/{r}')
    df_comb = pd.DataFrame()
    for p in pairs:    
        df_ = df[p]
        df_['Region'] = region
        df_['Price'] = df_.loc[:,p[1]]
        df_['Shock'] = p[1].split('_')[0]
        if len(p) == 2:
            df_['Shock Level'] = 'OK'
        else:
            df_['Shock Level'] = df_.loc[:,p[2]]    
            del(df_[p[2]])        
        del(df_[p[1]])
        df_comb = df_comb.append(df_)
    df_out = df_out.append(df_comb)

/anaconda3/envs/p37/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/anaconda3/envs/p37/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/anaconda3/envs/p37/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

In [121]:
df_out.shape

(120, 5)

In [122]:
df_out.head()

,time,Region,Price,Shock,Shock Level
0,Jul.17,Centrally Planned Asia,111.059250,Baseline,OK
1,Dec.17,Centrally Planned Asia,117.126270,Baseline,OK
2,Jul.18,Centrally Planned Asia,114.787445,Baseline,OK
3,Dec.18,Centrally Planned Asia,122.813370,Baseline,OK
4,Jul.19,Centrally Planned Asia,121.155840,Baseline,OK


In [123]:
df_out.groupby(['Shock Level'])['time'].count()

Shock Level
Critical    17
Extreme      3
Low         21
OK          79
Name: time, dtype: int64

In [124]:
df_out.groupby(["Region"]).Region.count()

Region
Centrally Planned Asia          24
European Union                  24
Former Soviet Union             24
Multiple Breadbasket Failure    24
North Atlantic Free Trade       24
Name: Region, dtype: int64

In [125]:
df_out[df_out['Shock'] != 'Baseline'].to_csv("TWIST.csv", index=False)